In [1]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')
# fashion_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TIL2020/fashion/TIL_NLP_train_dataset.csv')
fashion_train = pd.read_csv('datasets/TIL_NLP_train_dataset.csv')
fashion_train.head()

FileNotFoundError: [Errno 2] File datasets/TIL_NLP_train_dataset.csv does not exist: 'datasets/TIL_NLP_train_dataset.csv'

i fk you


In [ ]:
# take a look at data types in fashion_train
fashion_train.info()

In [ ]:
fashion_train

In [ ]:
# find the longest sentence

def max_len(x):
    a=x.split()
    return len(a)

print('maximum length of sentence = ', max( fashion_train['word_representation'].apply(max_len)))

In [ ]:
MAX_LEN = 47
DIM = 100

In [ ]:
# find the number of unique words

from keras.preprocessing.text import Tokenizer
vect=Tokenizer()
vect.fit_on_texts(fashion_train['word_representation'])
word_index = vect.word_index
print('Number of unique words= ', len(vect.word_index) + 1)

In [ ]:
fashion_test = pd.read_csv('datasets/TIL_NLP_test_dataset.csv')
fashion_test

In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM 
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.callbacks import ModelCheckpoint ,EarlyStopping
import pandas as pd
import numpy as np
import re
from keras.optimizers import Adam

import matplotlib.pyplot as plt

In [ ]:
# get output in another dataframe

train_labels_df = fashion_train[["outwear", "top", "trousers", "women dresses", "women skirts"]]
train_labels_df.head()

In [ ]:
# take a look at the distribution of output
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 10
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size

train_labels_df.sum(axis=0).plot.bar()

In [ ]:
# get X and y
X = []
sentences = list(fashion_train["word_representation"])
for sen in sentences:
    X.append(sen)

#Here we do not need to perform any one-hot encoding because our output labels are already in the form of one-hot encoded vectors.
y = train_labels_df.values

In [ ]:
y_train = y

In [ ]:
X_train = []
for index in fashion_train.index:
  X_train.append(fashion_train['word_representation'][index])
  
X_test = []
for index in fashion_test.index:
  X_test.append(fashion_test['word_representation'][index])

In [ ]:
#word embeddings

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

MAX_LEN = 47
DIM = 100
MAX_WORDS= 10000
VALIDATION_SPLIT = 0.2

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_LEN)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_LEN)


In [ ]:
#import the pretrained word embedding file

import pickle
with open('datasets/word_embeddings.pkl', 'rb') as f:
    word_embeddings = pickle.load(f)

In [ ]:
# processing word_embeddings.pkl as embedding matrix

print('Preparing embedding matrix.')



# prepare embedding matrix
num_words = min(MAX_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, DIM))
for word, i in word_index.items():
    if i >= MAX_WORDS:
        continue
    embedding_vector = word_embeddings.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_matrix.shape

In [ ]:
# Our model will have one input layer, one embedding layer, 2 LSTM layer with 128 neurons and one output layer with 5 neurons since we have 5 labels in the output

deep_inputs = Input(shape=(MAX_LEN,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128,return_sequences=True, dropout =0.4, recurrent_dropout = 0.4)(embedding_layer) #,return_sequences=True,,return_sequences=True, dropout =0.4, recurrent_dropout = 0.4
LSTM_Layer_2 = LSTM(64)(LSTM_Layer_1) #,dropout = 0.5, recurrent_dropout=0.5
# dense_layer_1 = Dense(16, activation='relu')(LSTM_Layer_2)
# dropout_1 = Dropout(0.5)(dense_layer_1)
dense_layer_2 = Dense(5, activation='sigmoid')(LSTM_Layer_2)
model = Model(inputs=deep_inputs, outputs=dense_layer_2)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
print(model.summary())

In [ ]:
es_callback = EarlyStopping(monitor='val_loss', patience=40)

In [ ]:
#visualisation of the above summary

from keras.utils import plot_model
plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=200, verbose=1, validation_split=0.2,callbacks=[es_callback])

In [ ]:
import matplotlib.pyplot as plt

history_dict = history.history

acc = history_dict['acc']
val_acc = history_dict['val_acc']
loss=history_dict['loss']
val_loss=history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()

# Processing Test Data

In [ ]:
X_test

In [ ]:
# predict crisp classes for test set
yhat = model.predict(X_test, verbose=0)
preds_labels = [[1 if x > 0.8 else 0 for idx,x in enumerate(i) ] for i in yhat]
preds_labels

In [ ]:
id_list = []
outwear = []
top = []
trousers = []
women_dresses =[]
women_skirts = []
for i in range(len(preds_labels)):
    id_list.append(i)
    outwear.append(preds_labels[i][0])
    top.append(preds_labels[i][1])
    trousers.append(preds_labels[i][2])
    women_dresses.append(preds_labels[i][3])
    women_skirts.append(preds_labels[i][4])


In [ ]:
output = pd.DataFrame({'id': id_list,
                       'outwear': outwear,
                       'top' : top,
                       'trousers': trousers,
                       'women dresses':women_dresses,
                       'women skirts': women_skirts})
output.to_csv('submission.csv', index=False)